In [1]:
# import the necessary libaries
import numpy as np  
import pandas as pd
from tensorflow import keras  
from keras.models import Sequential 
from keras.layers import Dense, Activation, SimpleRNN  
from keras.utils import to_categorical 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
# load the 'monthly-sunspots.csv' dataset
df = pd.read_csv('monthly-sunspots.csv')

# extract Year and Month from the 'Month' column
df['Year'] = df['Month'].str[:4].astype(int)
df['Month'] = df['Month'].str[5:7].astype(int)

# define the feature and targert variables
X = df[['Year', 'Month']]
y = df['Sunspots']

# Process the time-series dataset
X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X) 
y = y_scaler.fit_transform(y.values.reshape(-1, 1)) 

# data analysis step to find shape
print('Feature Shape:', X.shape)
print('Target Shape:', y.shape)

# reshape features to align with RNN model expectations (num_samples, time_steps, num_features)
X = X.reshape(X.shape[0], 1, X.shape[1])

# split the dataset into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Feature Shape: (2820, 2)
Target Shape: (2820, 1)


In [3]:
# set model parameters
input_shape = (12, X.shape[2])
units = 256
dropout = 0.2

# build the RNN model
model = Sequential(
    [
        SimpleRNN(units=units, dropout=dropout, input_shape=input_shape),
        Dense(1) # non-classficiation -> 1 dense layer
    ]
)

# compile the model with mean squared error loss, Adam optimizer, and mean squared error (MSE)
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

C:\Users\Jacob Klymenko\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [4]:
# define training parameters
batch_size = 128
epochs = 20

# train the model on the training data with the specified number of epochs and batch size
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

# evaluate the model of the test data
_, mse = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)

# print the MSE as a performance measure
print('\nTest Mean Squared Error (MSE): ', mse)

Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0319 - mse: 0.0319
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0297 - mse: 0.0297 
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0296 - mse: 0.0296 
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0292 - mse: 0.0292 
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0298 - mse: 0.0298 
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0288 - mse: 0.0288 
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0290 - mse: 0.0290 
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0318 - mse: 0.0318 
Epoch 9/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0288 - mse: 0.0288 
Epoch 10/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0296 - mse: 0.0296 
Epoch 11/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0307 - mse: 0.0307 
Epoch 12/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0308 - mse: 0.0308 
Epoch 13/20
18/18 ━━━━━━━━━━━━━━━━━━━━

In [5]:
# Results:
#
# This problem was also pretty interesting as the dataset did not contain the need for 
# classification prediction, but rather sunspots which look like continuous values. After 
# extrating the year and month from the first time-series column and scaling both of them 
# with the MinMaxScaler from Sklearn as mentioned in the problem statement, it was 
# interesting to work through how to adjust the RNN model to the shape of the dataset.
# However, testing different time step values for the input_shape variable overall did 
# not look like it affected the final performance of the mean squared error (MSE) value.
#
# Looking at the MSE value of approximately 0.028 with the understanding the sunspot ranges 
# from about 0 to 200, shows that the RNN model built in this file has a decent performance, as 
# it looks to be capturing some patterns throughout the dataset. However, it is not the best and 
# has room for improvement in reducing the error. I believe a of couple adjustment considerations 
# could be the time step, as for some reason it does not play a role here but should be, and the 
# data processing step is most likely incorrectly setting up the dataset for training. After 
# trying different hyper-parameters, this MSE value of about 0.028 is the best I can get. The 
# reason I chose MSE as both the loss function and metric is that the target variable, 'Sunspots', 
# is continuous rather than categorical.